In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
from keras.activations import softmax
import keras.backend as K
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
#m is no. of training examples, each example has a sequence of Tx inputs
Tx = 10
Ty = 10
frame_vals_len = 1805

In [3]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
activator = Activation(softmax,name='attention_weights')
dotor = Dot(axes = 1)
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
concatenator2 = Concatenate(axis=0)
densor = Dense(1, activation = "relu")
dotor = Dot(axes = 1)

In [4]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attetion) LSTM cell
    """
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor(concat)
    alphas = activator(e)
    context = dotor([alphas, a])
    
    return context

In [5]:
n_a = 64
n_s = 128
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(frame_vals_len, activation=softmax)

In [13]:
def model_fn(Tx, Ty, n_a, n_s, frame_vals_len):
    """
    Arguments:
    Tx -- length of the input sequence
    Ty -- length of the output sequence
    n_a -- hidden state size of the Bi-LSTM
    n_s -- hidden state size of the post-attention LSTM
    human_vocab_size -- size of the python dictionary "human_vocab"
    machine_vocab_size -- size of the python dictionary "machine_vocab"

    Returns:
    model -- Keras model instance
    """
    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx, frame_vals_len))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []

    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    for t in range(Ty):
        context = one_step_attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state = [s, c])
        out = output_layer(s)
        #out = tf.reshape(out,(1,1805))
        #print("out shape",out.shape)
        outputs.append(out)
    
    outputs2 = outputs[0]
    for softmax in range(1,10):
        outputs2 = concatenator([outputs2,outputs[softmax]])

    print(outputs2.shape)
    model = Model(inputs = [X, s0, c0], outputs = outputs2)
        
    return model

In [7]:
GRID_H = 19
GRID_W = 19
NO_OBJECT_SCALE  = 1.0
OBJECT_SCALE     = 5.0
COORD_SCALE      = 1.0
CLASS_SCALE      = 1.0

BATCH_SIZE       = 16
WARM_UP_BATCHES  = 0
TRUE_BOX_BUFFER  = 50

In [8]:
def custom_loss(y_true, y_pred):
    mask_shape = tf.shape(y_true)[:4]
    
    cell_x = tf.to_float(tf.reshape(tf.tile(tf.range(GRID_W), [GRID_H]), (1, GRID_H, GRID_W, 1, 1)))
    cell_y = tf.transpose(cell_x, (0,2,1,3,4))

    cell_grid = tf.tile(tf.concat([cell_x,cell_y], -1), [BATCH_SIZE, 1, 1, 1, 1])
    
    coord_mask = tf.zeros(mask_shape)
    conf_mask  = tf.zeros(mask_shape)
    class_mask = tf.zeros(mask_shape)
    
    seen = tf.Variable(0.)
    total_recall = tf.Variable(0.)
    
    """
    Adjust prediction
    """
    ### adjust x and y      
    pred_box_xy = tf.sigmoid(y_pred[..., 1:3]) + cell_grid
    
    ### adjust w and h
    pred_box_wh = tf.exp(y_pred[..., 3:5])
    
    ### adjust confidence
    pred_box_conf = tf.sigmoid(y_pred[..., 0])
    
    """
    Adjust ground truth
    """
    ### adjust x and y
    true_box_xy = y_true[..., 1:3] # relative position to the containing cell
    
    ### adjust w and h
    true_box_wh = y_true[..., 3:5] # number of cells accross, horizontally and vertically
    
    ### adjust confidence
    true_wh_half = true_box_wh / 2.
    true_mins    = true_box_xy - true_wh_half
    true_maxes   = true_box_xy + true_wh_half
    
    pred_wh_half = pred_box_wh / 2.
    pred_mins    = pred_box_xy - pred_wh_half
    pred_maxes   = pred_box_xy + pred_wh_half       
    
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_box_wh[..., 0] * true_box_wh[..., 1]
    pred_areas = pred_box_wh[..., 0] * pred_box_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)
    
    true_box_conf = iou_scores * y_true[..., 0]
    
    """
    Determine the masks
    """
    ### coordinate mask: simply the position of the ground truth boxes (the predictors)
    coord_mask = tf.expand_dims(y_true[..., 0], axis=-1) * COORD_SCALE
    
    ### confidence mask: penelize predictors + penalnp_arr_gt.shapeize boxes with low IOU
    # penalize the confidence of the boxes, which have IOU with some ground truth box < 0.6
    true_xy = true_boxes[..., 0:2]
    true_wh = true_boxes[..., 2:4]
    
    true_wh_half = true_wh / 2.
    true_mins    = true_xy - true_wh_half
    true_maxes   = true_xy + true_wnot outputs:h_half
    
    pred_xy = tf.expand_dims(pred_box_xy, 4)
    pred_wh = tf.expand_dims(pred_box_wh, 4)
    
    pred_wh_half = pred_wh / 2.
    pred_mins    = pred_xy - pred_wh_half
    pred_maxes   = pred_xy + pred_wh_half    
    np_arr_gt.shape
    intersect_mins  = tf.maximum(pred_mins,  true_mins)
    intersect_maxes = tf.minimum(pred_maxes, true_maxes)
    intersect_wh    = tf.maximum(intersect_maxes - intersect_mins, 0.)
    intersect_areas = intersect_wh[..., 0] * intersect_wh[..., 1]
    
    true_areas = true_wh[..., 0] * true_wh[..., 1]
    pred_areas = pred_wh[..., 0] * pred_wh[..., 1]

    union_areas = pred_areas + true_areas - intersect_areas
    iou_scores  = tf.truediv(intersect_areas, union_areas)

    best_ious = tf.reduce_max(iou_scores, axis=4)
    conf_mask = conf_mask + tf.to_float(best_ious < 0.6) * (1 - y_true[..., 4]) * NO_OBJECT_SCALE
    
    # penalize the confidence of the boxes, which are reponsible for corresponding ground truth box
    conf_mask = conf_mask + y_true[..., 4] * OBJECT_SCALE
          
    
    """
    Warm-up training
    """
    no_boxes_mask = tf.to_float(coord_mask < COORD_SCALE/2.)
    seen = tf.assign_add(seen, 1.)
    
    true_box_xy, true_box_wh, coord_mask = tf.cond(tf.less(seen, WARM_UP_BATCHES), 
                          lambda: [true_box_xy + (0.5 + cell_grid) * no_boxes_mask, 
                                   true_box_wh + tf.ones_like(true_box_wh) * np.reshape(ANCHORS, [1,1,1,BOX,2]) * no_boxes_mask, 
                                   tf.ones_like(coord_mask)],
                          lambda: [true_box_xy, 
                                   true_box_wh,
                                   coord_mask])
    
    """
    Finalize the loss
    """
    nb_coord_box = tf.reduce_sum(tf.to_float(coord_mask > 0.0))
    nb_conf_box  = tf.reduce_sum(tf.to_float(conf_mask  > 0.0))
    nb_class_box = tf.reduce_sum(tf.to_float(class_mask > 0.0))
    
    loss_xy    = tf.reduce_sum(tf.square(true_box_xy-pred_box_xy)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_wh    = tf.reduce_sum(tf.square(true_box_wh-pred_box_wh)     * coord_mask) / (nb_coord_box + 1e-6) / 2.
    loss_conf  = tf.reduce_sum(tf.square(true_box_conf-pred_box_conf) * conf_mask)  / (nb_conf_box  + 1e-6) / 2.
    loss = loss_xy + loss_wh + loss_conf + loss_class
    
    nb_true_box = tf.reduce_sum(y_true[..., 4])
    nb_pred_box = tf.reduce_sum(tf.to_float(true_box_conf > 0.5) * tf.to_float(pred_box_conf > 0.3))

    """
    Debugging code
    """    
    current_recall = nb_pred_box/(nb_true_box + 1e-6)
    total_recall = tf.assign_add(total_recall, current_recall) 

    loss = tf.Print(loss, [tf.zeros((1))], message='Dummy Line \t', summarize=1000)
    loss = tf.Print(loss, [loss_xy], message='Loss XY \t', summarize=1000)
    loss = tf.Print(loss, [loss_wh], message='Loss WH \t', summarize=1000)
    loss = tf.Print(loss, [loss_conf], message='Loss Conf \t', summarize=1000)
    loss = tf.Print(loss, [loss_class], message='Loss Class \t', summarize=1000)
    loss = tf.Print(loss, [loss], message='Total Loss \t', summarize=1000)
    loss = tf.Print(loss, [current_recall], message='Current Recall \t', summarize=1000)
    loss = tf.Print(loss, [total_recall/seen], message='Average Recall \t', summarize=1000)
    
    return loss

SyntaxError: invalid syntax (<ipython-input-8-92aeaf71e06e>, line 72)

In [8]:
def non_max_suppression_fast(boxes, overlapThresh):
    # if there are no boxes, return an empty list
    if len(boxes) == 0:
        return []
 
    # if the bounding boxes integers, convert them to floats --
    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")
        
    pick = []

    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
 
    # compute the area of the bounding boxes and sort the bounding boxes 
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(y2)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)
 
        # finding the largest (x, y) coordinates for the start of the bounding box and the smallest (x, y) coordinates for the end of the bounding box
        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])
        
        #width and height of the bounding box
        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)
        
        #ratio of overlap
        overlap = (w * h) / area[idxs[:last]]
        
        idxs = np.delete(idxs, np.concatenate(([last],
            np.where(overlap > overlapThresh)[0])))
 
    return pick

In [9]:
def del_conf_layer(x):
    return np.delete(x,0,axis = 1)

In [10]:
def my_loss(y_true, y_pred):
    """
    y_true: Array of shape(?,10,19,19,5) containing ground truth values for each sequence
    y_pred: Array of shape(?,10,19,19,5) containing predicted values from final softmax layer for each sequence
    """
    y_shape0 = K.shape(y_pred)[0]
    y_pred = tf.reshape(y_pred,(K.shape(y_pred)[0],10,19,19,5))
    y_true = tf.reshape(y_pred,(y_shape0,10,19,19,5))
    #print(y_pred.get_shape().as_list()[1:5])
    lambda_coord = 5
    lambda_noobj = 0.5
    #suppression of extremely less chance boxes
    threshold = 0.5
    int_mask = threshold*np.ones(y_pred.get_shape().as_list()[1:5])
    #int_mask = threshold*np.ones(y_pred.get_shape().as_list(),tf.float32)
    int_mask[...,1:5] = np.zeros((y_pred.get_shape().as_list()[1:5])[:3]+[4])
    #int_mask[...,1:5] = tf.zeros((y_pred.get_shape().as_list(),tf.float32)[1:4]+[4])
    int_mask = tf.convert_to_tensor(int_mask,dtype = tf.float32)
    caster = tf.cast((y_pred-int_mask)>0,dtype = tf.float32)
    mul_mask = np.ones(y_pred.get_shape().as_list()[1:5])
    #mul_mask = tf.ones(y_pred.get_shape().as_list(),tf.float32)
    mul_mask[...,1:5] = np.zeros((y_pred.get_shape().as_list()[1:5])[:3]+[4])
    mul_mask = tf.convert_to_tensor(mul_mask,dtype = tf.float32)
    caster = caster*mul_mask
    caster = 0.5*caster
    y_pred = tf.nn.relu(y_pred - int_mask)
    y_pred = y_pred + caster
    y_pred_new = np.zeros(y_pred.get_shape().as_list()[1:5])
    y_pred_new[...,1:5] = np.ones((y_pred.get_shape().as_list()[1:5])[:3]+[4])
    #non max suppression
    
    #selecting boxes and formatting for passing into nms
    for i in range(Tx):
        index_mask = np.zeros(y_pred.shape[2:])
        index_mask[...,0] = np.reshape(np.arange(19*19)+1,(19,19))
        boxes_selector = tf.where(y_pred[y_shape0,i]>0,tf.convert_to_tensor(index_mask),tf.convert_to_tensor(np.zeros(y_pred.shape[2:])))
        box_locations = tf.contrib.layers.dense_to_sparse(boxes_selector).values
        boxes_dims = []
        coordinates_init = []
        scores = []
        for j in range(19*19):
            try:
                midpoint_index = box_locations[j]
            except IndexError:
                break
            coordinate_y = tf.cast(tf.floor((midpoint_index-1)/19),tf.float32)
            coordinate_y_int = tf.cast(coordinate_y,tf.int32)
            coordinate_x = tf.cast((midpoint_index-1)%19,tf.float32)
            coordinate_x_int = tf.cast((midpoint_index-1)%19,tf.int32)
            score = y_pred[y_shape0,i,coordinate_x_int,coordinate_y_int,0]
            xyhw = [(1/19)*(coordinate_x+y_pred[y_shape0,i,coordinate_x_int,coordinate_y_int,1]), (1/19)*(coordinate_y+y_pred[y_shape0,i,coordinate_x_int,coordinate_y_int,2]),(1/19)*(y_pred[y_shape0,i,coordinate_x_int,coordinate_y_int,3]),(1/19)*(y_pred[y_shape0,i,coordinate_x_int,coordinate_y_int,4])]
            yx2 = [xyhw[1]-(xyhw[2]/2),xyhw[0]-(xyhw[3]/2),xyhw[1]+(xyhw[3]/2),xyhw[0]+(xyhw[3]/2)]
            boxes_dims.append(yx2)
            coordinates_init.append(midpoint_index)
            scores.append(score)
        selected_indices = tf.image.non_max_suppression(boxes=boxes_dims,scores = scores,max_output_size=tf.constant(2),iou_threshold=threshold)
        coordinates_init = tf.convert_to_tensor(coordinates_init)
        index_arr = np.reshape(np.arange(19*19),(19,19))
        
        for k in range(19*19):
            try:
                index = selected_indices[k]
            except IndexError:
                break
            mid = coordinates_init[index]
            #new_x = tf.cast(coords[0],tf.int32)
            #new_y = tf.cast(coords[1],tf.int32)
            
            y_pred_new = y_pred_new + tf.where(index_mask == mid,mul_mask,np.zeros(y_pred.get_shape().as_list()[1:5])) 
            #y_pred_new[i,k,new_x,new_y,0] = 1
            
    y_pred = y_pred*y_pred_new        
    
    #calculating loss using yolo loss function
    pred_box_xy = tf.sigmoid(y_pred[..., 1:3])
    pred_box_hw = tf.exp(y_pred[..., 3:5])
    pred_box_conf = tf.sigmoid(y_pred[..., 0])
    
    true_box_xy = y_true[...,1:3]
    true_box_hw = y_pred[...,3:5]
    true_box_conf = y_pred[...,0]
    
    loss_xy = lambda_coord*np.sum(true_box_conf*np.square(true_box_xy-pred_box_xy))
    loss_wh = lambda_coord*np.sum(true_box_conf*np.square(np.sqrt(true_box_hw)-np.sqrt(pred_box_hw)))
    loss_conf = np.sum(true_box_conf*np.square(true_box_conf - pred_box_conf)) + lambda_noobj*np.sum(np.abs(true_box_conf-1)*np.square(true_box_conf - pred_box_conf))
    loss = loss_xy + loss_wh + loss_conf
    #debugging
    loss = tf.Print(loss, [tf.zeros((1))], message='Dummy Line \t', summarize=1000)
    loss = tf.Print(loss, [loss_xy], message='Loss XY \t', summarize=1000)
    loss = tf.Print(loss, [loss_wh], message='Loss WH \t', summarize=1000)
    loss = tf.Print(loss, [loss_conf], message='Loss Conf \t', summarize=1000)
    loss = tf.Print(loss, [loss], message='Total Loss \t', summarize=1000)
    return loss

In [ ]:
 """ 
    for i in range(Tx):
        index_of_boxes = []
        boxes_dims =[]
        for j in range(GRID_H):
            for k in range(GRID_W):
                if y_pred[K.shape(y_pred)[0],i,j,k,0]>0 is not None:
                    index_of_boxes.append([i,j,k])
                    xyhw = [(1/19)*(j+y_pred[i,j,k,1]), (1/19)*(k+y_pred[i,j,k,2]),(1/19)*(y_pred[i,j,k,3]),(1/19)*(y_pred[i,j,k,4])]
                    xy2 = [xywh[0]-(xywh[3]/2),xywh[1]-(xywh[2]/2),xywh[0]+(xywh[3]/2),xywh[1]+(xywh[3]/2)]
                    boxes_dims.append(xy2)
        picked_boxes_indices = non_max_suppression_fast(boxes_dims, threshold)
        for index in picked_boxes_indices:
            picked_box = boxes_dims[index]
            new_xyhw = [(picked_box[2] - picked_box[0])/2,(picked_box[3] - picked_box[1])/2,picked_box[3] - picked_box[1], picked_box[2] - picked_box[0]]
            y_pred_new[i,int(new_xywh[0]/(1/19)),int(new_xywh[1]/(1/19)),1] = new_xyhw[0]/(1/19) - int(new_xyhw[0]/(1/19))
            y_pred_new[i,int(new_xywh[0]/(1/19)),int(new_xywh[1]/(1/19)),2] = new_xyhw[1]/(1/19) - int(new_xyhw[1]/(1/19))
            y_pred_new[i,int(new_xywh[0]/(1/19)),int(new_xywh[1]/(1/19)),3] = new_xyhw[2]/(1/19)
            y_pred_new[i,int(new_xywh[0]/(1/19)),int(new_xywh[1]/(1/19)),4] = new_xyhw[3]/(1/19)
        y_pred = tf.convert_to_tensor(y_pred_new)
    """    

In [23]:
a = np.reshape(0.5*np.arange(1000),(10,5,5,4))
mask = np.ones((10,5,5,4))
#a = tf.convert_to_tensor(a,dtype = tf.float32)
mask[...,1:4] = np.zeros((10,5,5,3))
mask = mask/2
mask = tf.convert_to_tensor(mask,dtype = tf.float32)
a = tf.convert_to_tensor(a,dtype = tf.float32)
#a = tf.cast(a >= 5.5,dtype = np.int32)
np_a = tf.Session().run(a)
b = np.zeros(a.shape)
b[...,0] = np.reshape(np.arange(250),(10,5,5))
c = tf.where(a>5,tf.convert_to_tensor(b),tf.convert_to_tensor(np.zeros(a.shape)))
#c_sparse = tf.contrib.layers.dense_to_sparse(c).values
d = tf.zeros([1,1])
#print(np_a[1,1,2,1])
x_coord = tf.constant(1)
y_coord = tf.constant(2)
var = tf.Variable(tf.ones((10,5,5,4),tf.float32))
var[1,x_coord,y_coord,1].assign(0)
init_op = tf.global_variables_initializer()
tf.Session().run(init_op)

#print(x_coord)
#print(np_a[1,x_coord,y_coord,1])
#tf.Session().run(c_sparse)
#tf.where(a>5.5,tf.convert_to_tensor(b),tf.convert_to_tensor(np.zeros(a.shape)))

In [14]:
model = model_fn(Tx, Ty, n_a, n_s, 1805)

(?, 18050)


In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 10, 1805)     0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 10, 128)      957440      input_2[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 10, 128)      0           s0[0][0]                         
                                                                 lstm_1[10][0]                    
          

In [17]:
out = model.compile(optimizer=Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01),
                    loss=my_loss)
out


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



ValueError: Dimensions must be equal, but are 10 and 19 for 'loss/concatenate_2_loss/mul_14444' (op: 'Mul') with input shapes: [?,10,19,19], [?,10,19,19,2].

# 